## A minimalist example for recovering sparse graphs using `mGLAD`

Fitting meta-GLAD on a batch of erdos-renyi random sparse graphs with samples obtained from a corresponding multivariate Gaussian distribution  

### About `mGLAD` 
A meta learning based approach to recover sparse graphs. This work proposes `mGLAD` which is a unsupervised version of a previous `GLAD` model (GLAD: Learning Sparse Graph Recovery (ICLR 2020 - [link](<https://openreview.net/forum?id=BkxpMTEtPB>)).  

Key Benefits & features:  
- It is a fast alternative to solving the Graphical Lasso problem as
    - GPU based acceleration can be leveraged
    - Requires less number of iterations to converge due to neural network based acceleration of the unrolled optimization algorithm (Alternating Minimization).     
- mGLAD automatically learns the sparsity related regularization parameters. This gives an added benefit to mGLAD over other graphical lasso solvers.  
- The meta loss is the logdet objective of the graphical lasso `1/B(-1*log|theta|+ <S, theta>)`, where `B=batch_size, S=input covariance matrix, theta=predicted precision matrix`.   

In [2]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

/home/harshx/anaconda3/envs/dagM


In [3]:
import torch
torch.__version__

'1.9.0'

### Create Synthetic data

In [4]:
from scripts import main
# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, trueTheta = main.getGLADdata(
    num_nodes=10, 
    sparsity=0.2, 
    num_samples=500, 
    batch_size=2
)    
print(f'TrueTheta: {trueTheta.shape}, Samples {Xb.shape}')

TrueTheta: (2, 10, 10), Samples (2, 500, 10)


### Running the GLAD-Meta algorithm
Until Convergence:  

    1. Initialize learnable `GLAD` parameters
    2. Run the GLAD model
    3. Get the meta-loss
    4. Backprop

Possible reasons if `mGLAD` does not converge: 

    1. Please re-run. This will run the optimization with different initializations  
    2. Lower the learning rate  
    3. Change the INIT_DIAG=0/1 in the `GLAD` model parameters
    4. Increase the theta_init_offset value (should be >0.1)

In [14]:
# main source file for meta-GLAD

# Helper functions for DAG Meta
from scripts.glad.glad_params import glad_params
from scripts.glad import glad
from scripts.main import lossGLADmeta
from scripts.utils import prepare_data
from scripts.utils.metrics import reportMetrics

from pprint import pprint

def initGLAD():
    """
    Initialize the GLAD model parameters and the optimizer
    to be used.
    """
    model = glad_params(theta_init_offset=1, nF=3, H=3)
    optimizer = glad.get_optimizers(model)
    return model, optimizer

def runGLAD(Sb, model_glad):
    """Run the input through the GLAD meta algorithm.
    It executes the following steps in batch mode
    1. Run the GLAD model to get initial good regularization
    2. Calculate the meta-loss
    
    Args:
        Sb (torch.Tensor BxDxD): The input covariance matrix
        metaGLADmodel (dict): Contains the learnable params
    
    Returns:
        loss (torch.scalar): The meta loss 
        predTheta (torch.Tensor BxDxD): The predicted theta
    """
    # 1. Running the GLAD model 
    predTheta = glad.glad(Sb, model_glad)
    # 2. Calculate the meta-loss
    loss = lossGLADmeta(predTheta, Sb)
    return loss, predTheta


def GLAD_Meta_main(Xb, trueTheta=None, EPOCHS=250):
    """Running the DAG Meta algorithm.
    
    Args:
        Xb (torch.Tensor BxMxD): The input sample matrix
        trueDAG (torch.Tensor BxDxD): The corresponding 
            true DAGs for reporting metrics.
        EPOCHS (int): The number of training epochs
        
    """
    # Calculating the batch covariance
    Sb = prepare_data.getCovariance(Xb) # BxDxD
    # Converting the data to torch 
    Xb = prepare_data.convertToTorch(Xb, req_grad=False)
    Sb = prepare_data.convertToTorch(Sb, req_grad=False)
    trueTheta = prepare_data.convertToTorch(
        trueTheta,
        req_grad=False
        )
    B, M, D = Xb.shape
    # optimizer and model for GLAD
    model_glad, optimizer_glad = initGLAD()
    # Optimizing for the meta loss
    for e in range(EPOCHS):      
        # reset the grads to zero
        optimizer_glad.zero_grad()
        # calculate the loss
        loss, predTheta = runGLAD(Sb, model_glad)
        # calculate the backward gradients
        loss.backward()
        if not e%25: print(f'epoch:{e}/{EPOCHS} loss:{loss.detach().numpy()}')
        # updating the optimizer params with the grads
        optimizer_glad.step()
        # reporting the metrics if true DAGs provided
        if trueTheta is not None and (e+1)%EPOCHS == 0:
            for b in range(B):
                compare_theta = reportMetrics(
                    trueTheta[b].detach().numpy(), 
                    predTheta[b].detach().numpy()
                )
                print(f'Batch:{b} - {compare_theta}')

In [15]:
GLAD_Meta_main(Xb, trueTheta)

epoch:0/250 loss:14.798260688781738
epoch:25/250 loss:14.316488265991211
epoch:50/250 loss:13.990367889404297
epoch:75/250 loss:13.66386604309082
epoch:100/250 loss:12.982595443725586
epoch:125/250 loss:9.633712768554688
epoch:150/250 loss:9.097713470458984
epoch:175/250 loss:8.861564636230469
epoch:200/250 loss:8.712800979614258
epoch:225/250 loss:8.62151050567627
Batch:0 - {'FDR': 0.16666666666666666, 'TPR': 1.0, 'FPR': 0.05714285714285714, 'SHD': 2, 'nnzTrue': 10, 'nnzPred': 12, 'precsion': 0.8333333333333334, 'recall': 1.0, 'Fbeta': 0.9090909090909091, 'aupr': 1.0, 'auc': 1.0}
Batch:1 - {'FDR': 0.0, 'TPR': 1.0, 'FPR': 0.0, 'SHD': 0, 'nnzTrue': 5, 'nnzPred': 5, 'precsion': 1.0, 'recall': 1.0, 'Fbeta': 1.0, 'aupr': 1.0, 'auc': 1.0}
